In [2]:
!pip install --q refractml

ERROR: snowflake-snowpark-python 1.0.0 has requirement snowflake-connector-python<3.0.0,>=2.7.12, but you'll have snowflake-connector-python 3.6.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement snowflake-snowpark-python<2,>=1.4.0, but you'll have snowflake-snowpark-python 1.0.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jupyterlab-server 2.25.4 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install --q xgboost

ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement snowflake-snowpark-python<2,>=1.4.0, but you'll have snowflake-snowpark-python 1.0.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install --q catboost

ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement snowflake-snowpark-python<2,>=1.4.0, but you'll have snowflake-snowpark-python 1.0.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: openapi-spec-validator 0.7.1 has requirement jsonschema<5.0.0,>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.2.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; pyth

In [1]:
from snowflake.snowpark import Session
import configparser

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
import gc

In [3]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
NFOLDS = 3
SEED = 0
NROWS = None

# Code to establish connection and read data from Snowflake

In [5]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [6]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [7]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [8]:
application_train_sf  = session.table("CRA_APPLICATION_TRAIN_DETAILS")
application_test_sf  = session.table("CRA_APPLICATION_TEST_DETAILS")
previous_application_sf  = session.table("CRA_PREVIOUS_APPLICATION_DETAILS")

# Convert Snowflake data into Pandas dataframes

In [9]:
data = application_train_sf.to_pandas()
test = application_test_sf.to_pandas()
prev = previous_application_sf.to_pandas()

In [10]:
data.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
test.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)
prev.drop(['CREATED_BY','CREATED_AT'], axis=1, inplace=True)

In [11]:
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

In [12]:
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

In [13]:
gc.enable()

y_train = data['TARGET']
del data['TARGET']

In [14]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

In [15]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [16]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test= x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [17]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

In [18]:
x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

In [33]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

    def predict_proba(self, x):
        return self.clf.predict_proba(x)[:,1]

In [34]:
class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)[:,1]

In [35]:
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))
    
    def predict_proba(self, x):
        return self.gbdt.predict_proba(xgb.DMatrix(x))[:,1]

In [22]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [78]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
    'random_state':0
}

In [79]:
rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
    'random_state':0
}

In [80]:
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

In [81]:
catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False,
    'random_seed':0
}

In [27]:
#xg = XgbWrapper(seed=SEED, params=xgb_params)
#et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
#rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
#cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)

# XGBoost Classifier

In [61]:
#xg_oof_train, xg_oof_test = get_oof(xg)

In [62]:
param = xgb_params
param['seed'] = 0
nrounds = 250

dtrain = xgb.DMatrix(x_train, label=y_train)
gbdt = xgb.train(param, dtrain, nrounds)

/tmp/pip_packages/xgboost/core.py:160: UserWarning: [09:48:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
#y_prob = gbdt.predict(xgb.DMatrix(x))
#y_pred = np.round(y_prob)

In [64]:
y_prob = gbdt.predict(xgb.DMatrix(x_train))

In [65]:
y_pred = np.round(y_prob)

In [66]:
y_prob

array([0.0574903 , 0.0402197 , 0.08202588, ..., 0.07238798, 0.04120503,
       0.116495  ], dtype=float32)

In [67]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [75]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    y_pred = model.predict(xgb.DMatrix(data))
    #y_prob = model.predict_proba(xgb.DMatrix(data))[:,1]
    temp_dict = {"Prediction: ": np.round(y_pred), "Probability: ": y_pred }
    return temp_dict

In [69]:
payload  = x_train.iloc[0].to_dict()
payload

{'NAME_CONTRACT_TYPE_x': 0.0,
 'CODE_GENDER': 0.0,
 'FLAG_OWN_CAR': 0.0,
 'FLAG_OWN_REALTY': 0.0,
 'CNT_CHILDREN': 1.0,
 'AMT_INCOME_TOTAL': 270000.0,
 'AMT_CREDIT_x': 1288350.0,
 'AMT_ANNUITY_x': 37669.5,
 'AMT_GOODS_PRICE_x': 1125000.0,
 'NAME_TYPE_SUITE_x': 0.0,
 'NAME_INCOME_TYPE': 0.0,
 'NAME_EDUCATION_TYPE': 0.0,
 'NAME_FAMILY_STATUS': 0.0,
 'NAME_HOUSING_TYPE': 0.0,
 'REGION_POPULATION_RELATIVE': 0.02461,
 'DAYS_BIRTH': -13625.0,
 'DAYS_EMPLOYED': -1725.0,
 'DAYS_REGISTRATION': -3376.0,
 'DAYS_ID_PUBLISH': -5593.0,
 'OWN_CAR_AGE': 0.0,
 'FLAG_MOBIL': 1.0,
 'FLAG_EMP_PHONE': 1.0,
 'FLAG_WORK_PHONE': 1.0,
 'FLAG_CONT_MOBILE': 1.0,
 'FLAG_PHONE': 0.0,
 'FLAG_EMAIL': 0.0,
 'OCCUPATION_TYPE': 0.0,
 'CNT_FAM_MEMBERS': 3.0,
 'REGION_RATING_CLIENT': 2.0,
 'REGION_RATING_CLIENT_W_CITY': 2.0,
 'WEEKDAY_APPR_PROCESS_START_x': 0.0,
 'HOUR_APPR_PROCESS_START_x': 14.0,
 'REG_REGION_NOT_LIVE_REGION': 0.0,
 'REG_REGION_NOT_WORK_REGION': 0.0,
 'LIVE_REGION_NOT_WORK_REGION': 0.0,
 'REG_CITY_NOT_L

In [70]:
print ('{ "payload": ', payload, "}")

{ "payload":  {'NAME_CONTRACT_TYPE_x': 0.0, 'CODE_GENDER': 0.0, 'FLAG_OWN_CAR': 0.0, 'FLAG_OWN_REALTY': 0.0, 'CNT_CHILDREN': 1.0, 'AMT_INCOME_TOTAL': 270000.0, 'AMT_CREDIT_x': 1288350.0, 'AMT_ANNUITY_x': 37669.5, 'AMT_GOODS_PRICE_x': 1125000.0, 'NAME_TYPE_SUITE_x': 0.0, 'NAME_INCOME_TYPE': 0.0, 'NAME_EDUCATION_TYPE': 0.0, 'NAME_FAMILY_STATUS': 0.0, 'NAME_HOUSING_TYPE': 0.0, 'REGION_POPULATION_RELATIVE': 0.02461, 'DAYS_BIRTH': -13625.0, 'DAYS_EMPLOYED': -1725.0, 'DAYS_REGISTRATION': -3376.0, 'DAYS_ID_PUBLISH': -5593.0, 'OWN_CAR_AGE': 0.0, 'FLAG_MOBIL': 1.0, 'FLAG_EMP_PHONE': 1.0, 'FLAG_WORK_PHONE': 1.0, 'FLAG_CONT_MOBILE': 1.0, 'FLAG_PHONE': 0.0, 'FLAG_EMAIL': 0.0, 'OCCUPATION_TYPE': 0.0, 'CNT_FAM_MEMBERS': 3.0, 'REGION_RATING_CLIENT': 2.0, 'REGION_RATING_CLIENT_W_CITY': 2.0, 'WEEKDAY_APPR_PROCESS_START_x': 0.0, 'HOUR_APPR_PROCESS_START_x': 14.0, 'REG_REGION_NOT_LIVE_REGION': 0.0, 'REG_REGION_NOT_WORK_REGION': 0.0, 'LIVE_REGION_NOT_WORK_REGION': 0.0, 'REG_CITY_NOT_LIVE_CITY': 0.0, 'REG_

In [76]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(gbdt, y_req)

{'Prediction: ': array([0.], dtype=float32),
 'Probability: ': array([0.0574903], dtype=float32)}

In [77]:
## registering the model in refract.
model_reg = register_model(gbdt,
               score, 
               name="CRA_XGBoost_Model", 
               description="Credit Risk XGBoost Model",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_train,
               y_pred=y_pred,
               prob=y_prob,
               features=x_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_train, 
               y_train=y_train.tolist(),
               y_test=y_train.tolist(),
               feature_names=x_train.columns.tolist(),
               original_features=x_train.columns.tolist(),
               feature_ids=x_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# RandomForest Classifier

In [83]:
RFC = RandomForestClassifier(**rf_params)
RFC.fit(x_train, y_train)

RandomForestClassifier(max_depth=12, max_features=0.2, min_samples_leaf=2,
                       n_estimators=200, n_jobs=16, random_state=0)

In [85]:
y_pred = RFC.predict(x_train)

In [86]:
y_prob = RFC.predict_proba(x_train)[:,1]

In [87]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [88]:
y_prob

array([0.05953402, 0.04038833, 0.05930782, ..., 0.06358363, 0.04462819,
       0.15885422])

In [92]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    y_pred = model.predict(data)
    y_prob = model.predict_proba(data)[:,1]
    temp_dict = {"Prediction: ": np.round(y_pred), "Probability: ": y_prob }
    return temp_dict

In [90]:
payload  = x_train.iloc[0].to_dict()
print ('{ "payload": ', payload, "}")

{ "payload":  {'NAME_CONTRACT_TYPE_x': 0.0, 'CODE_GENDER': 0.0, 'FLAG_OWN_CAR': 0.0, 'FLAG_OWN_REALTY': 0.0, 'CNT_CHILDREN': 1.0, 'AMT_INCOME_TOTAL': 270000.0, 'AMT_CREDIT_x': 1288350.0, 'AMT_ANNUITY_x': 37669.5, 'AMT_GOODS_PRICE_x': 1125000.0, 'NAME_TYPE_SUITE_x': 0.0, 'NAME_INCOME_TYPE': 0.0, 'NAME_EDUCATION_TYPE': 0.0, 'NAME_FAMILY_STATUS': 0.0, 'NAME_HOUSING_TYPE': 0.0, 'REGION_POPULATION_RELATIVE': 0.02461, 'DAYS_BIRTH': -13625.0, 'DAYS_EMPLOYED': -1725.0, 'DAYS_REGISTRATION': -3376.0, 'DAYS_ID_PUBLISH': -5593.0, 'OWN_CAR_AGE': 0.0, 'FLAG_MOBIL': 1.0, 'FLAG_EMP_PHONE': 1.0, 'FLAG_WORK_PHONE': 1.0, 'FLAG_CONT_MOBILE': 1.0, 'FLAG_PHONE': 0.0, 'FLAG_EMAIL': 0.0, 'OCCUPATION_TYPE': 0.0, 'CNT_FAM_MEMBERS': 3.0, 'REGION_RATING_CLIENT': 2.0, 'REGION_RATING_CLIENT_W_CITY': 2.0, 'WEEKDAY_APPR_PROCESS_START_x': 0.0, 'HOUR_APPR_PROCESS_START_x': 14.0, 'REG_REGION_NOT_LIVE_REGION': 0.0, 'REG_REGION_NOT_WORK_REGION': 0.0, 'LIVE_REGION_NOT_WORK_REGION': 0.0, 'REG_CITY_NOT_LIVE_CITY': 0.0, 'REG_

In [93]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(RFC, y_req)

{'Prediction: ': array([0], dtype=int8), 'Probability: ': array([0.05953402])}

In [94]:
## registering the model in refract.
model_reg = register_model(RFC,
               score, 
               name="CRA_RandomForest_Model", 
               description="Credit Risk RandomForest Model",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_train,
               y_pred=y_pred,
               prob=y_prob,
               features=x_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_train, 
               y_train=y_train.tolist(),
               y_test=y_train.tolist(),
               feature_names=x_train.columns.tolist(),
               original_features=x_train.columns.tolist(),
               feature_ids=x_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# Extra Tree Classifier

In [95]:
ETC = ExtraTreesClassifier(**et_params)
ETC.fit(x_train, y_train)

ExtraTreesClassifier(max_depth=12, max_features=0.5, min_samples_leaf=2,
                     n_estimators=200, n_jobs=16, random_state=0)

In [96]:
y_pred = ETC.predict(x_train)
y_prob = ETC.predict_proba(x_train)[:,1]

In [97]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [98]:
y_prob

array([0.06163793, 0.03988874, 0.05960309, ..., 0.09010163, 0.0393106 ,
       0.21154737])

In [99]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    y_pred = model.predict(data)
    y_prob = model.predict_proba(data)[:,1]
    temp_dict = {"Prediction: ": np.round(y_pred), "Probability: ": y_prob }
    return temp_dict

In [100]:
payload  = x_train.iloc[0].to_dict()
print ('{ "payload": ', payload, "}")

{ "payload":  {'NAME_CONTRACT_TYPE_x': 0.0, 'CODE_GENDER': 0.0, 'FLAG_OWN_CAR': 0.0, 'FLAG_OWN_REALTY': 0.0, 'CNT_CHILDREN': 1.0, 'AMT_INCOME_TOTAL': 270000.0, 'AMT_CREDIT_x': 1288350.0, 'AMT_ANNUITY_x': 37669.5, 'AMT_GOODS_PRICE_x': 1125000.0, 'NAME_TYPE_SUITE_x': 0.0, 'NAME_INCOME_TYPE': 0.0, 'NAME_EDUCATION_TYPE': 0.0, 'NAME_FAMILY_STATUS': 0.0, 'NAME_HOUSING_TYPE': 0.0, 'REGION_POPULATION_RELATIVE': 0.02461, 'DAYS_BIRTH': -13625.0, 'DAYS_EMPLOYED': -1725.0, 'DAYS_REGISTRATION': -3376.0, 'DAYS_ID_PUBLISH': -5593.0, 'OWN_CAR_AGE': 0.0, 'FLAG_MOBIL': 1.0, 'FLAG_EMP_PHONE': 1.0, 'FLAG_WORK_PHONE': 1.0, 'FLAG_CONT_MOBILE': 1.0, 'FLAG_PHONE': 0.0, 'FLAG_EMAIL': 0.0, 'OCCUPATION_TYPE': 0.0, 'CNT_FAM_MEMBERS': 3.0, 'REGION_RATING_CLIENT': 2.0, 'REGION_RATING_CLIENT_W_CITY': 2.0, 'WEEKDAY_APPR_PROCESS_START_x': 0.0, 'HOUR_APPR_PROCESS_START_x': 14.0, 'REG_REGION_NOT_LIVE_REGION': 0.0, 'REG_REGION_NOT_WORK_REGION': 0.0, 'LIVE_REGION_NOT_WORK_REGION': 0.0, 'REG_CITY_NOT_LIVE_CITY': 0.0, 'REG_

In [101]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(ETC, y_req)

{'Prediction: ': array([0], dtype=int8), 'Probability: ': array([0.06163793])}

In [ ]:
## registering the model in refract.
model_reg = register_model(ETC,
               score, 
               name="CRA_ExtraTree_Model", 
               description="Credit Risk ExtraTreeClassifier Model",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_train,
               y_pred=y_pred,
               prob=y_prob,
               features=x_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_train, 
               y_train=y_train.tolist(),
               y_test=y_train.tolist(),
               feature_names=x_train.columns.tolist(),
               original_features=x_train.columns.tolist(),
               feature_ids=x_train.columns,
               kyd=True, kyd_score = True)

# CATBoost Classifier

In [ ]:
CBC = CatBoostClassifier(**catboost_params)
CBC.fit(x_train, y_train)

In [ ]:
y_pred = CBC.predict(x_train)
y_prob = CBC.predict_proba(x_train)[:,1]

In [ ]:
y_pred

In [ ]:
y_prob

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    y_pred = model.predict(data)
    y_prob = model.predict_proba(data)[:,1]
    temp_dict = {"Prediction: ": np.round(y_pred), "Probability: ": y_prob }
    return temp_dict

In [ ]:
payload  = x_train.iloc[0].to_dict()
print ('{ "payload": ', payload, "}")

In [ ]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(CBC, y_req)

In [ ]:
## registering the model in refract.
model_reg = register_model(CBC,
               score, 
               name="CRA_CATBoost_Model", 
               description="Credit Risk CATBoost Model",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_train,
               y_pred=y_pred,
               prob=y_prob,
               features=x_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_train, 
               y_train=y_train.tolist(),
               y_test=y_train.tolist(),
               feature_names=x_train.columns.tolist(),
               original_features=x_train.columns.tolist(),
               feature_ids=x_train.columns,
               kyd=True, kyd_score = True)

# STACKING Models

In [ ]:
x_train_XGB = XGB.predict(xgb.DMatrix(x_train))
x_train_RFC = RFC.predict_proba(x_train)
x_train_ETC = ETC.predict_proba(x_train)
x_train_CBC = CBC.predict_proba(x_train)

In [ ]:
x_test_XGB = XGB.predict(xgb.DMatrix(x_test))
x_test_RFC = RFC.predict_proba(x_test)
x_test_ETC = ETC.predict_proba(x_test)
x_test_CBC = CBC.predict_proba(x_test)

In [102]:
x_train = np.concatenate((x_train_XGB, x_train_RFC, x_train_ETC, x_train_CBC), axis=1)
x_test = np.concatenate((x_test_XGB, x_test_RFC, x_test_ETC, x_test_CBC), axis=1)

In [103]:
print("{},{}".format(x_train.shape, x_test.shape))

(307511, 4),(48744, 4)


In [119]:
RFC_Stack = RandomForestClassifier()
RFC_Stack.fit(x_train,y_train)

RandomForestClassifier()

In [120]:
y_pred = RFC_Stack.predict(x_train)
y_prob = RFC_Stack.predict_proba(x_train)

In [124]:
y_pred

24825

In [121]:
y_prob

24803

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    
    data_XGB = XGB.predict(xgb.DMatrix(data))
    data_RFC = RFC.predict_proba(data)
    data_ETC = ETC.predict_proba(data)
    data_CBC = CBC.predict_proba(data)
    
    data = np.concatenate((data_XGB, data_RFC, data_ETC, data_CBC), axis=1)
    
    y_pred = model.predict(data)
    y_prob = model.predict_proba(data)[:,1]
    temp_dict = {"Prediction: ": np.round(y_pred), "Probability: ": y_prob }
    return temp_dict

In [ ]:
payload  = x_train.iloc[0].to_dict()
print ('{ "payload": ', payload, "}")

In [ ]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
score(RFC_Stack, y_req)

In [ ]:
## registering the model in refract.
model_reg = register_model(RFC_Stack,
               score, 
               name="CRA_Stacked_Model", 
               description="Credit Risk Stacked RandomForest Model",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="pip install snowflake-ml-python==1.0.11",
               y_true=y_train,
               y_pred=y_pred,
               prob=y_prob,
               features=x_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_train, 
               y_train=y_train.tolist(),
               y_test=y_train.tolist(),
               feature_names=x_train.columns.tolist(),
               original_features=x_train.columns.tolist(),
               feature_ids=x_train.columns,
               kyd=True, kyd_score = True)